<a href="https://colab.research.google.com/github/gyesibiney/Digital-mapping/blob/main/digital_mapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# mounting drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
#import library
from google.colab import auth
auth.authenticate_user()
import pandas as pd
import folium
import gspread
from google.auth import default



In [3]:
# data loading
data= pd.read_csv('/content/drive/MyDrive/sample bus route coordinates - Sheet1.csv')

In [4]:
data.head()

,coordinates,latitude,longitude
0,"43.253320172012685, -79.86928707988771",43.253320,-79.869287
1,"43.25063682142333, -79.85209349199839",43.250637,-79.852093
2,"43.247623452723595, -79.8386634532602",43.247623,-79.838663
3,"43.25606256486144, -79.8328625962096",43.256063,-79.832863
4,"43.26472809952085, -79.86688950030884",43.264728,-79.866890


In [5]:
# Retrieve credentials and ignore the optional second value (denoted by _)
# The credentials will be used for authenticating with Google Cloud services
credentials,_= default()

In [6]:
# Authorize the application using the provided credentials
# This creates an authorized connection to Google Sheets
gs= gspread.authorize(credentials)

In [7]:
# creating a new worksheet in googlesheet
spreadsheet=gs.create('coordinate_data')

In [8]:
# getting the first sheet of the created worksheet
worksheet= spreadsheet.get_worksheet(0)

In [9]:
# update the worksheet with data
worksheet.update([data.columns.tolist()]+ data.values.tolist())

{'spreadsheetId': '1sZws47Slttc9PeGHTK1CC-oFmtLM2VdC2ld3zSBZ01I',
 'updatedRange': 'Sheet1!A1:C8',
 'updatedRows': 8,
 'updatedColumns': 3,
 'updatedCells': 24}

In [10]:
# Create a folium map centered at the specified location with a zoom level of 12
# The 'location' parameter specifies the center of the map with latitude and longitude
# The 'zoom_start' parameter specifies the initial zoom level of the map
pt= folium.Map(location=[43.253320172012685, -79.86928707988771],zoom_start=12)

In [11]:
#Extract latitude and longitude from the 'coordinates' column and create new columns 'pt_lat' and 'pt_lng'
data[['pt_lat', 'pt_lng']]=data.coordinates.str.split(',',expand= True).astype(float).values.tolist()

In [12]:
data.pt_lat

0    43.253320
1    43.250637
2    43.247623
3    43.256063
4    43.264728
5    43.230531
6    43.241457
Name: pt_lat, dtype: float64

In [13]:
data.pt_lng

0   -79.869287
1   -79.852093
2   -79.838663
3   -79.832863
4   -79.866890
5   -79.870163
6   -79.910718
Name: pt_lng, dtype: float64

In [14]:
# Create an empty list to store latitude and longitude pairs
points = []

# Iterate over the rows of the DataFrame
for i in range(len(data.pt_lat)):
    # Append a list containing latitude and longitude to the 'points' list
    points.append([data.pt_lat[i], data.pt_lng[i]])


# Add markers to the map
for index, latitude in enumerate(data['pt_lat']):
    folium.Marker([latitude, data['pt_lng'][index]],
                  popup='Bus station{} \n' .format(index),
                  icon=folium.Icon(color='red', icon_color='blue', prefix='fa', icon='bus')).add_to(pt)

# Create a polyline
points = list(zip(data['pt_lat'], data['pt_lng']))
folium.PolyLine(locations=points, color='red', dash_array='5', opacity=0.85, tooltip='transit_route').add_to(pt)

# Save the map as an HTML file
pt.save('bus_stations_map.html')

# Display the map
pt

In [15]:
from geopy.distance import geodesic

# Create a function to calculate distance between two points using Haversine formula
def calculate_distance(point1, point2):
    return geodesic(point1, point2).miles

# Create a new column 'distance' to store the distance at each transit route
data['distance'] = 0.0  # Initialize with 0.0

# Iterate over rows and calculate distance for each row
for i in range(1, len(data)):
    # Get the coordinates of two consecutive points
    coords1 = (data['pt_lat'][i - 1], data['pt_lng'][i - 1])
    coords2 = (data['pt_lat'][i], data['pt_lng'][i])

    # Calculate the distance using the Haversine formula
    distance = calculate_distance(coords1, coords2)

    # Update the 'distance' column with the calculated distance
    data.at[i, 'distance'] = distance


In [16]:
data.head()

,coordinates,latitude,longitude,pt_lat,pt_lng,distance
0,"43.253320172012685, -79.86928707988771",43.253320,-79.869287,43.253320,-79.869287,0.000000
1,"43.25063682142333, -79.85209349199839",43.250637,-79.852093,43.250637,-79.852093,0.887138
2,"43.247623452723595, -79.8386634532602",43.247623,-79.838663,43.247623,-79.838663,0.708914
3,"43.25606256486144, -79.8328625962096",43.256063,-79.832863,43.256063,-79.832863,0.651977
4,"43.26472809952085, -79.86688950030884",43.264728,-79.866890,43.264728,-79.866890,1.817989
